Задачи
Импортируйте библиотеку pandas как pd. Загрузите датасет bookings.csv с разделителем ;. Проверьте размер таблицы, типы переменных, а затем выведите первые 7 строк, чтобы посмотреть на данные. 
Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.


Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
На сколько ночей в среднем бронируют отели разных типов?
Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?
Проанализируйте даты запланированного прибытия. – На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов
Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?
Создайте колонку total_kids, объединив children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?
Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.
 

Описание данных
Имеются следующие переменные:

Hotel – тип отеля (City Hotel или Resort Hotel)  
Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
Arrival full date – полная дата прибытия
Arrival date year – год прибытия  
Arrival date month – месяц прибытия  
Arrival date week number – номер недели прибытия
Arrival date day of month – день прибытия
Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
Adults – число взрослых
Children – число детей
Babies – число младенцев 
Meal – выбранный тип питания
Country – страна происхождения клиента
Reserved room type – тип зарезервированного номера
Assigned room type – тип полученного номера (может отличаться от забронированного)
Customer type – тип бронирования
Reservation status – значение последнего статуса брони: Canceled - было отменено клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не зарегистрировался и сообщил администрации отеля причину
Reservation status date – дата обновления статуса

In [3]:
import pandas as pd

In [4]:
path = '~/Projects_Python/Source/2_bookings.csv' # set the file path

In [5]:
df = pd.read_csv(path, sep=';')# read first 10 rows in file using separation between columns as ";"

In [ ]:
#let's rename columns in a common way

In [ ]:
rename_cloumns = {}
for col in df.columns:
    rename_cloumns[col] = col.replace(" ", "_").lower()



In [ ]:
df = df.rename(columns = rename_cloumns)

In [ ]:
df.dtypes # check data types in table

In [ ]:
#there are dates columns but now the type is wrong

In [ ]:
df[["arrival_full_date","reservation_status_date"]] = df[["arrival_full_date","reservation_status_date"]].apply(pd.to_datetime)

TOP-5 countries according to succesfull  booking amount

In [ ]:
df.query('reservation_status == "Check-Out"')\
.groupby("country", as_index = False)\
.agg({"hotel":"count"})\
.rename(columns ={"hotel":"qty"})\
.sort_values(by ="qty", ascending = False)\
.head(5)

In [ ]:
#df.loc[df.is_canceled==0].country.value_counts().head(5)

What is the average qty of nights in bookings in the different types of hotels?   

In [ ]:
df['ttl_days']=df['stays_in_weekend_nights'] + df['stays_in_week_nights'] # calculate ttl numner of week and weekend days

In [ ]:
av_nights = df.groupby("hotel", as_index = False).agg({"ttl_days":"mean"}).round()
av_nights

Sometimes assigned_room_type is not the same as reserved_room_type. It may happen in case of overbooking.
How many of such cases are in df? What dates?

In [ ]:
len(df.loc[df.assigned_room_type != df.reserved_room_type])#calculate the number of rows where the values are not same

In [ ]:
df.query('assigned_room_type != reserved_room_type')\
.groupby(["arrival_full_date","hotel"], as_index = False)\
.agg({"country":"count"})\
.rename(columns = {"country":"qty"})\
.sort_values(by ="qty", ascending = False)\
.head(10) #top 10 results when the room type was not the same as it was planned, when it happened and where(hotel)

What are the planned arrival dates? What month was the top in 2016? Is it same in 2017?

In [ ]:
df.query('arrival_date_year == ["2016", "2017"]').groupby(["arrival_date_year",'arrival_date_month']).agg({"hotel":"count"})\
.rename(columns = {"hotel":"qty"})\
.sort_values(by ="qty", ascending = False)\
.head()

Group data by hotels, which month was the most cancelled for City Hotel.

In [ ]:
df.query('is_canceled == "1"')\
.groupby(['hotel','arrival_date_month','arrival_date_year'])\
.agg({"country":"count"})\
.rename(columns = {"country":"qty"})\
.sort_values(by = ["hotel","qty"], ascending = False)

In [ ]:
#another way
df.groupby(["arrival_date_year",'arrival_date_month']).is_canceled.sum().groupby('arrival_date_year').nlargest(1)


Look at the numerical characteristics of three variables: adults, children and babies. Which one has the largest average value?
Create a total_kids column by concatenating children and babies. For which type of hotels did the average value of the variable turn out to be the largest?

In [ ]:
df[['children','babies','adults']].mean()

In [ ]:
df['total_kids']= df['children'] + df['babies'] #create new col with info from 'children' and 'babies'

In [ ]:
df.groupby(['hotel']).total_kids.mean()


Create a has_kids variable that takes the value True if the client specified at least one child during the booking (total_kids), otherwise False. Calculate the ratio of the number of users who left to the total number of customers, expressed as a percentage (churn rate). Indicate which group has the highest rate.

In [ ]:
def has_kids(x): #function that checks id the value > 0 
    if x > 0:
     return True
    else: 
     return False

In [ ]:
df['has_kids']= df['total_kids'].apply(has_kids)# create new col and applying created function has_kids

In [ ]:
#df['has_kids']=df.total_kids.astype(bool)

In [ ]:
df.total_kids.astype(bool).value_counts()

In [ ]:
rates = df.groupby('has_kids')['is_canceled'].value_counts(normalize=True)
rates 